In [2]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your api key: ")

Enter your api key:  ········


In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [4]:
response=llm.invoke('Who won the last valorant championship?')
print(response)

content='The last Valorant Championship was the Valorant Champions 2022, which took place from August 31 to September 18, 2022. The winner of the championship was OpTic Gaming! They defeated LOUD in the finals with a score of 4-3, taking home the championship title and the prize of $350,000.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 18, 'total_tokens': 90, 'completion_time': 0.232101282, 'prompt_time': 0.00117419, 'queue_time': 0.01227453, 'total_time': 0.233275472}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_87cbfbbc4d', 'finish_reason': 'stop', 'logprobs': None} id='run-a1d7873c-ed24-4b52-9c33-fbc5d40661a2-0' usage_metadata={'input_tokens': 18, 'output_tokens': 72, 'total_tokens': 90}


In [5]:
import chromadb
chroma_client = chromadb.Client()

In [6]:
collection = chroma_client.create_collection(name="my_collection")

In [7]:
collection.add(
    documents=[
        "This is a document about New York",
        "This is a document about Delhi"
    ],
    ids=["id1", "id2"]
)


In [8]:
all_docs=collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'metadatas': [None, None],
 'documents': ['This is a document about New York',
  'This is a document about Delhi'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [ ]:
collection.query(
    query_texts=['Query is about India'],
    n_results=2
)


In [18]:
details=collection.get()

In [19]:
collection.delete(ids=details['ids'])

In [20]:
collection.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [22]:
os.environ['USER_AGENT'] = 'myagent'

In [32]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/88533478290662086-software-development-manager-generative-ai-google-workspace")
page_data=loader.load().pop().page_content
print(page_data)

Software Development Manager, Generative AI, Google Workspace — Google CareersCareersSkip to main contentTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend Feedbackjob detailsarrow_backBack to jobs searchJobs search results2,788  jobs matchedCustomer Engineering Manager II, Startups, Google CloudSunnyvale, CA, USA; San Francisco, CA, USAChip Package Signal and Power Integrity EngineerSunnyvale, CA, USACloud Consultant, Application Modernization Edge, Professional ServicesAustin, TX, USA; Atlanta, GA, USA; +2 more; +1 moreSoftware Engineer, Content Safety On Device, Hades PlatformBelo Horizonte, State of Minas Gerais, Brazil; São Paulo, State of São Paulo, BrazilQuantum Error Correction Research Scientist, Quantum AILos Angeles, CA, USA; Goleta, CA, USAAccount Strategist, Engage, Google Customer SolutionsDublin, Irela

In [37]:
from langchain_core.prompts import PromptTemplate

example_prompt=PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}

    ### INSTRUCTION:
    The scraped text is from the careers page of a website.
    Your job is to extract the job posting and return them in JSON format containing following roles:
    `role`,`experience`,`skills` and `description`. 
    Only return the valid JSON
    ### VALID JSON ONLY (NO PREAMBLE): 
    """
)

chain_extract=example_prompt| llm
res=chain_extract.invoke(input={'page_data':page_data})
print(res.content)

[
    {
        "role": "Software Development Manager, Generative AI, Google Workspace",
        "experience": "8 years of experience with software development in one or more programming languages, 3 years of experience in a technical leadership role, 2 years of experience in a people management, supervision/team leadership role",
        "skills": "Generative AI, machine learning algorithms and tools, artificial intelligence, deep learning, natural language processing",
        "description": "Set and communicate team priorities that support the broader organization's goals. Align strategy, processes, and decision-making across teams. Develop the mid-term technical vision and roadmap within the scope of your team. Design, guide and vet systems designs within the scope of the broader area, and write product or system development code to solve ambiguous problems."
    }
]


In [40]:
type(res.content)

str

In [76]:
from langchain_core.output_parsers import JsonOutputParser
json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

[{'role': 'Software Development Manager, Generative AI, Google Workspace',
  'experience': '8 years of experience with software development in one or more programming languages, 3 years of experience in a technical leadership role, 2 years of experience in a people management, supervision/team leadership role',
  'skills': 'Generative AI, machine learning algorithms and tools, artificial intelligence, deep learning, natural language processing',
  'description': "Set and communicate team priorities that support the broader organization's goals. Align strategy, processes, and decision-making across teams. Develop the mid-term technical vision and roadmap within the scope of your team. Design, guide and vet systems designs within the scope of the broader area, and write product or system development code to solve ambiguous problems."}]

In [78]:
json_res=json_res[0]

In [79]:
type(json_res)

dict

In [80]:
json_res

{'role': 'Software Development Manager, Generative AI, Google Workspace',
 'experience': '8 years of experience with software development in one or more programming languages, 3 years of experience in a technical leadership role, 2 years of experience in a people management, supervision/team leadership role',
 'skills': 'Generative AI, machine learning algorithms and tools, artificial intelligence, deep learning, natural language processing',
 'description': "Set and communicate team priorities that support the broader organization's goals. Align strategy, processes, and decision-making across teams. Develop the mid-term technical vision and roadmap within the scope of your team. Design, guide and vet systems designs within the scope of the broader area, and write product or system development code to solve ambiguous problems."}

In [46]:
import pandas as pd
df=pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [88]:
import chromadb
import uuid
client=chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row['Techstack'],
                       metadatas={'links':row['Links']},
                       ids=[str(uuid.uuid4())])

In [89]:
data=collection.get()
print(data)

{'ids': ['0059927f-2cc5-4e25-b02d-dc2b0f8b177b', '15d340d5-82c7-4a8e-bf53-e96415b79f7e', '4b88590c-6d2c-4bcb-a7dc-65e8ad4cb228', '4d7c026d-693e-4959-9e45-c527e5f9336c', '5a90dffe-e15b-4cf4-b095-d6baeb55fbc4', '5f64fb06-2522-4d4d-8eb9-11b1c98ce9c9', '6ff825c5-95ff-4ee4-b43b-262d047f7640', '735fe5ba-04e6-4de3-b68e-18421c7509c8', '7a1a117d-cf2c-47e2-b2d0-58552d87c110', '7bf6b538-4e95-4b29-b206-c80fda82abf4', '868966e7-154c-4821-8107-63e89c2aaa3f', '9bbb7bb3-1c89-43e7-926c-e3e32c47fe70', '9c465825-1092-437a-9661-0cc9f802747f', 'c9dd12b0-c108-4968-861e-0c9bda83ffc0', 'cfd91a15-a530-45d7-8226-cafe5585951f', 'd0ff565a-791a-4f20-9a6f-00c2f5b54daf', 'f0a2e833-a4d6-4dc9-9b73-5c9c4dc9a847', 'f7d1ce9e-304e-48d1-9e96-8a6b79bf57b0', 'fb18dd6a-e0c1-486c-8c14-c8f9dbcd93cd', 'fe840a81-c9cf-45ff-a4aa-1637cb049718'], 'embeddings': None, 'metadatas': [{'links': 'https://example.com/react-native-portfolio'}, {'links': 'https://example.com/full-stack-js-portfolio'}, {'links': 'https://example.com/kotlin-and

In [90]:
links=collection.query(query_texts=[json_res['skills']],n_results=2)['metadatas']

In [91]:
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [92]:
job=json_res

In [93]:
json_res['skills']

'Generative AI, machine learning algorithms and tools, artificial intelligence, deep learning, natural language processing'

In [94]:
prompt_email=PromptTemplate.from_template(
    '''
    ### JOB_DESCRIPTION:
    {job_description}
    ### INSTRUCTION:
    You are Siddharth, a student from final year BTech -MTech(Integrated) Computer Science student at LNMIIT Jaipur college who is well versed
    in Android Development, Web Development and even Full stack development and is currently working on use of llms.
    During my coding journey you have won various acheievements and been a part of Google Summer of code 2023 at NRNB worked under an Harvard Professor
    and also interned at Deloitte which is a big four.
    Your job is to write a cold email to the recruiter regarding the job mentioned above in fullfiling their needs.
    Also add the most relevant links from the following to showcase the work done by you: {portfolio_links}
    Remember you are Siddharth a prefinal student.
    DO NOT PROVIDE PREAMBLE
    ### EMAIL (NO PREAMBLE):
    
    '''
)
email_chain= prompt_email | llm
res=email_chain.invoke(input={'job_description':str(job),"portfolio_links":links})
print(res.content)

Here is the cold email:

Subject: Application for Software Development Manager, Generative AI, Google Workspace

Dear Hiring Manager,

I'm excited to apply for the Software Development Manager, Generative AI, Google Workspace role at Google. As a final year BTech-MTech (Integrated) Computer Science student at LNMIIT Jaipur, I'm confident that my skills and experience make me a strong fit for this position.

With a strong foundation in software development, I've explored various areas, including Android Development, Web Development, and Full Stack Development. Currently, I'm working on the application of Large Language Models (LLMs), which aligns with the requirements of this role. My experience in machine learning algorithms and tools, artificial intelligence, deep learning, and natural language processing will enable me to make significant contributions to the team.

As a technical leader, I've had the opportunity to work on multiple projects, including my participation in Google Summ